In [52]:
from crm import Network

In [1]:
import re
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import trange

In [2]:
import dill
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        dill.dump(obj, outp, dill.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return dill.load(inp)

# Data

In [3]:
# https://stackoverflow.com/questions/4666973/how-to-extract-the-substring-between-two-markers

In [4]:
f = open("data/sample1/raw/rand_crm_1.pl")

In [5]:
edges_raw = []

In [6]:
while True:
    s = f.readline()
    if not s:
        break
    if "connected" in s:
        edges_raw.append(s)

In [7]:
edges_raw[24]

'connected(a(998),[a(36)],[1]).\n'

In [8]:
edges_raw[24].split(",")[0].split("(")[-1].split(")")[0]

'998'

In [9]:
gg = re.search("\[a(\(.+?\))\]", edges_raw[-2])

In [10]:
gg.group(1)

'(0),a(0)'

In [11]:
edges_raw[24]

'connected(a(998),[a(36)],[1]).\n'

In [12]:
edges_raw[-2]

'connected(a(2),[a(0),a(0)],[1,0]).\n'

In [13]:
re.search("\[(a.+?)\]", edges_raw[-2]).group(1)

'a(0),a(0)'

In [14]:
re.search("\[(.*?)\]", edges_raw[-2]).group(1)

'a(0),a(0)'

In [15]:
edges_raw[-2], edges_raw[2]

('connected(a(2),[a(0),a(0)],[1,0]).\n', 'connected(a(1020),[a(30)],[1]).\n')

In [16]:
re.search("a\((.*)\),\[a\((.*?)\),a\((.*?)\)\]", edges_raw[-2]).group(3)

'0'

In [17]:
re.search("a\((.*)\),\[a\((.*?)\)\]", edges_raw[2]).group(0)

'a(1020),[a(30)]'

In [18]:
edges_raw[-1]

'connected(a(1),[a(0),a(0)],[1,0]).\n'

In [145]:
edges = []
itere = 0
for i in range(len(edges_raw)):
    
    # Binary Operators
    b_match = re.search("a\((.*)\),\[a\((.*?)\),a\((.*?)\)\]", edges_raw[i])
    u_match = re.search("a\((.*)\),\[a\((.*?)\)\]", edges_raw[i])
    
    end, start_one, start_two = -1, -1, -1
    
    if b_match:
        end, start_one, start_two = int(b_match.group(1))-1, int(b_match.group(2))-1, int(b_match.group(3))-1
    else:
        end, start_one = int(u_match.group(1))-1, int(u_match.group(2))-1
    # start_one --> end
    # start_two --> end
    
    if start_one == start_two and start_one!=-1:
        edges.append((int(start_one), int(end)))
    else:
        if start_one!=-1:
            edges.append((int(start_one), int(end)))
        if start_two!=-1:
            edges.append((int(start_two), int(end)))

In [20]:
# func(f(872),[type=complex,def=(train(_137465):-has_car(_137465,_137471),wheels(_137471,3),has_car(_137465,_137483),long(_137483),has_car(_137465,_137494),wheels(_137494,3),has_car(_137465,_137506),long(_137506))]).
# func(f(50),[type=complex,def=(train(_137465):-has_car(_137465,_137471),wheels(_137471,3),has_car(_137465,_137483),long(_137483))]).

In [146]:
with open("data/sample1/edges.txt", "w") as fp:
    for u, v in edges:
        fp.write(f"{u} {v}\n")

In [147]:
len(edges), len(set(edges))

(1114, 1114)

In [200]:
X_train = []
y_train = []

In [201]:
f = open("data/sample1/raw/rand_crm_1_train_features_pos")

In [202]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e) for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(1022)}
    X_train.append(dd)
    y_train.append(torch.tensor(1))

In [203]:
f = open("data/sample1/raw/rand_crm_1_train_features_neg")

In [204]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e) for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(1022)}
    X_train.append(dd)
    y_train.append(torch.tensor(0))

In [205]:
X_test = []
y_test = []

In [206]:
# !cat data/sample1/raw/rand_crm_1_test_features_pos

In [207]:
f = open("data/sample1/raw/rand_crm_1_test_features_pos")

In [208]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e) for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(1022)}
    X_test.append(dd)
    y_test.append(torch.tensor(1))

In [209]:
f = open("data/sample1/raw/rand_crm_1_test_features_neg")

In [210]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e) for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(1022)}
    X_test.append(dd)
    y_test.append(torch.tensor(0))

In [158]:
num_neurons = max([max(u, v) for u, v in edges])+1

In [159]:
num_neurons

1022

In [160]:
adj_list = [[] for i in range(num_neurons)]

In [161]:
len(adj_list)

1022

In [162]:
for u, v in edges:
    adj_list[u].append(v)

In [163]:
n = Network(num_neurons, adj_list)

In [164]:
orig_output_neurons = n.output_neurons

In [165]:
len(orig_output_neurons)

963

In [166]:
adj_list.append([])
adj_list.append([])

In [167]:
num_neurons = len(adj_list)

In [168]:
num_neurons

1024

In [169]:
for i in range(num_neurons):
    if i in orig_output_neurons:
        adj_list[i].append(num_neurons-2)
        adj_list[i].append(num_neurons-1)

In [170]:
adj_list[213]

[1022, 1023]

In [171]:
n = Network(num_neurons, adj_list)

In [172]:
n.set_neuron_activation(1022, lambda x : x, lambda x : 1)

In [173]:
n.set_neuron_activation(1023, lambda x : x, lambda x : 1)

In [174]:
F.cross_entropy

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Union[torch.Tensor, NoneType] = None, size_average: Union[bool, NoneType] = None, ignore_index: int = -100, reduce: Union[bool, NoneType] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

In [183]:
n.reset()
# X_train[2] = {i: 1 for i in range(len(X_train[2])+2)}
X_train[145][1022]=1
X_train[145][1023]=1
n.forward(X_train[145])

tensor([[2.8682],
        [3.0079]], grad_fn=<StackBackward0>)

In [116]:
# X_train[2]

In [117]:
sum([n.neurons[i].value for i in range(n.num_neurons)])

tensor([0.], grad_fn=<AddBackward0>)

In [118]:
n = Network(2, [[1], []])
n.forward({0:1, 1:1})

tensor([[0.4049]], grad_fn=<StackBackward0>)

In [122]:
len(adj_list)

1024

In [128]:
n = Network(1024, adj_list)

In [129]:
ip = {i: 1 for i in range(1024)}
n.forward(ip)

tensor([[0.],
        [0.]], grad_fn=<StackBackward0>)

In [185]:
n.neurons[12].value

tensor(1)

In [186]:
n.neurons[6]

6: 0	 Grad: 0
            
Predecessor: []	Successor: [944, 744, 287, 274, 85, 67, 59, 31]

In [187]:
n.neurons[1021]

1021: tensor([0.], grad_fn=<MulBackward0>)	 Grad: 0
            
Predecessor: [59]	Successor: [1022, 1023]

In [188]:
n.neurons[59]

59: tensor([0.], grad_fn=<MulBackward0>)	 Grad: 0
            
Predecessor: [6, 20]	Successor: [1021, 947, 931, 894, 887, 798, 782, 636, 616, 593, 586, 550, 541, 517, 515, 350, 328, 286, 257, 255, 220, 188, 187, 184, 156, 128, 125, 93, 87, 73]

In [189]:
X_train[0][202]

0

In [190]:
for i in range(len(X_train)):
    X_train[i][1022]=1
    X_train[i][1023]=1

In [191]:
for i in range(len(X_test)):
    X_train[i][1022]=1
    X_train[i][1023]=1

In [211]:
assert len(X_train)==len(y_train)

In [214]:
assert len(X_test)==len(y_test)

In [192]:
save_object(adj_list, "data/sample1/adj_list.dill")

In [215]:
save_object(X_train, "data/sample1/X_train.dill")
save_object(X_test, "data/sample1/X_test.dill")
save_object(X_test, "data/sample1/y_train.dill")
save_object(X_test, "data/sample1/y_test.dill")